In [114]:
import torch
import torch.nn as nn
import torch.cuda as cuda
# Load in relevant libraries, and alias where appropriate
import torchvision
import torchvision.transforms as transforms
import tensorflow as tf
import math
import numpy as np
import torch
import torch.nn.functional as F
from torch.nn import Parameter
from torch.nn.modules.conv import _ConvNd, init
from torch.nn.modules.utils import _single, _pair, _triple

In [115]:
def zeropoint_quantize(X,W):

    tensor_input = torch.tensor(X)
    tensor_weight = torch.tensor(W)
    #getting Mins of each array
    Mins=torch.empty(2)
    #print("Mins=",Mins)
    Mins[0]=torch.tensor(torch.min(tensor_input))
    Mins[1]=torch.tensor(torch.min(tensor_weight))

    #print("Mins=",Mins)

    #getting Maxs of each array
    Maxs=torch.empty(2)
    #print("Maxs=",Maxs)
    Maxs[0]=torch.tensor(torch.max(tensor_input))
    Maxs[1]=torch.tensor(torch.max(tensor_weight))

    #print("Maxs=",Maxs)


    # Calculate value range (denominator)
    x_range = torch.max(Maxs) - torch.min(Mins)
    #print("x_range=",x_range)
    x_range = 1 if x_range == 0 else x_range

    # Calculate scale
    scale = 255 / x_range
    #print("scale=",scale)


    # Shift by zero-point
    zeropoint = (-scale * torch.min(Mins)).round()


    # Scale and round the inputs
    X_quant = torch.clip((X * scale + zeropoint).round(), 0, 255)
    W_quant = torch.clip((W * scale + zeropoint).round(), 0, 255)


    #print("X_quant=",X_quant)
    #print("X_quant=",W_quant)

    return X_quant.to(torch.int32),W_quant.to(torch.int32),scale,zeropoint

In [116]:
def zeropoint_dequantize(X,z,scale,sumation):

    # Dequantize
    X_dequant = (X - (z*z)-(scale*z*sumation)) / (scale*scale)

    #print("X_dequant=",X_dequant)


    return  X_dequant

In [117]:
# Define relevant variables for the ML task
batch_size = 64
num_classes = 10
learning_rate = 0.001
num_epochs = 4

# Device will determine whether to run the training on GPU or CPU.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


#Loading the dataset and preprocessing
train_dataset = torchvision.datasets.MNIST(root = './data',
                                           train = True,
                                           transform = transforms.Compose([
                                                  transforms.Resize((32,32)),
                                                  transforms.ToTensor(),
                                                  transforms.Normalize(mean = (0.1307,), std = (0.3081,))]),
                                           download = True)


test_dataset = torchvision.datasets.MNIST(root = './data',
                                          train = False,
                                          transform = transforms.Compose([
                                                  transforms.Resize((32,32)),
                                                  transforms.ToTensor(),
                                                  transforms.Normalize(mean = (0.1325,), std = (0.3105,))]),
                                          download=True)


train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)


test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)


class nnModel(nn.Module):
    def __init__(self, num_classes):
        super(nnModel, self).__init__()

        self.conv1 =  nn.Conv2d(1, 6, kernel_size=5)
        self.relu1=nn.ReLU()
        self.maxpool1=nn.MaxPool2d(kernel_size = 2, stride = 2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.relu2=nn.ReLU()
        self.maxpool2=nn.MaxPool2d(kernel_size = 2, stride = 2)
        self.conv3 = nn.Conv2d(16, 120, kernel_size=5)

        self.fc1 = nn.Linear(120, 84)
        self.relu4 = nn.ReLU()
        self.fc2 = nn.Linear(84, num_classes)

    def forward(self, x):

        out = self.conv1(x)
        out = self.relu1(out)
        out = self.maxpool1(out)
        out = self.conv2(out)
        out = self.relu2(out)
        out = self.maxpool2(out)
        out = self.conv3(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.relu4(out)
        out = self.fc2(out)

        return out


In [118]:
# Load the saved model weights
model = nnModel(num_classes)

In [119]:
print(model.state_dict().keys())

odict_keys(['conv1.weight', 'conv1.bias', 'conv2.weight', 'conv2.bias', 'conv3.weight', 'conv3.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias'])


In [120]:
model.load_state_dict(torch.load('/content/drive/MyDrive/lenet-weight/weightsLenetPytorch3Conv-2.pt'))

<All keys matched successfully>

In [121]:
layer_weights1 = model.state_dict()['conv1.weight']
layer_bias1 = model.state_dict()['conv1.bias']
layer_weights2 = model.state_dict()['conv2.weight']
layer_bias2 = model.state_dict()['conv2.bias']
layer_weights3 = model.state_dict()['conv3.weight']
layer_bias3 = model.state_dict()['conv3.bias']
layer_weightsfc1 = model.state_dict()['fc1.weight']
layer_biasfc1 = model.state_dict()['fc1.bias']
layer_weightsfc2 = model.state_dict()['fc2.weight']
layer_biasfc2 = model.state_dict()['fc2.bias']
#print("layer_weights1=",layer_weights1)
#print("layer_bias1=",layer_bias1)
#print("layer_weights2=",layer_weights2)
#print("layer_bias2=",layer_bias2)
#print("layer_weights3=",layer_weights3)
#print("layer_bias3=",layer_bias3)
print("layer_weightsfc1=",layer_weightsfc1.shape)
print("layer_biasfc1=",layer_biasfc1.shape)
print("layer_weightsfc2=",layer_weightsfc2.shape)
print("layer_biasfc2=",layer_biasfc2.shape)
#print(layer_weights.shape)

layer_weightsfc1= torch.Size([84, 120])
layer_biasfc1= torch.Size([84])
layer_weightsfc2= torch.Size([10, 84])
layer_biasfc2= torch.Size([10])


In [122]:
layer_weights1.shape,layer_weights2.shape,layer_weights3.shape

(torch.Size([6, 1, 5, 5]),
 torch.Size([16, 6, 5, 5]),
 torch.Size([120, 16, 5, 5]))

In [123]:
class FCLayer1(nn.Linear):
    def __init__(self, n_inputs, n_outputs):
        super(FCLayer1, self).__init__(n_inputs, n_outputs)
        print("n_inputsfc1=",n_inputs)
        print("n_outputsfc1=",n_outputs)
        self.n_inputs = n_inputs
        self.n_outputs = n_outputs

        # Initialize weights and biases as PyTorch tensors
        self.weights = layer_weightsfc1#nn.Parameter(torch.randn(n_outputs, n_inputs))
        self.biases =layer_biasfc1# nn.Parameter(torch.zeros(n_outputs))

    def forward(self, input_data):
        #////////////////////////////Skippy Multiplicant///////////////////////////////////
        XBit=8

        def UpDownCounter(Enable,x,IncDec):
          IntX=int(x)

          if Enable != 0:
            if IntX == 1:
              if IncDec == 1:
                IntX=1
              else:
                IntX=-1
            else:
              IntX=0
          else:
            IntX=0
          return IntX
        #/////////////////////////////////////////////////////////////////////////////////////////////////////
        def BISC(In):
          LenSc= 2 ** XBit

          sc=np.zeros(LenSc)

          for x in range(XBit):
            i=(2 ** x)-1
            while i<LenSc:
              sc[i] = In[x]
              i+= 2 ** (x+1)

          sc = np.flip(sc)
          return sc

        #/////////////////////////////////////////////////////////////////////////////////////////////////////
        def DecimalToBinary(num,XBit):
          i = 0
          bnum = []
          Binary=''
          while num!=0:
            rem = num%2
            bnum.insert(i, rem)
            i = i+1
            num = int(num/2)
          i = i-1
          while i>=0:
            Binary=Binary+str(bnum[i])
            i = i-1

          while len(Binary)<XBit:
            Binary='0'+Binary

          return Binary

        #///////////////////////////////////////////////////////////////////////////////////////////////////

        output=torch.zeros((input_data.shape[0],self.n_outputs))
        #print("shapeoutputfc1=",output.shape)

        for batch in range(input_data.shape[0]):

            for y in range(self.n_outputs):
                #print("input_data",input_data)
                #print("input_data[batch,:]",input_data[batch,:])
                #print("self.weights",self.weights)
                #print("self.weights[y,:]",self.weights[y,:])

                X=input_data[batch,:]
                W=self.weights[y,:]
                #///////////////////////////Quantize input & Weight//////////////////////////////
                summation= torch.flatten(X + W)
                Xquantize,Wquantize,scale,zeropoint=zeropoint_quantize(X,W)
                #print("Xquantize",Xquantize)
                #print("Wquantize",Wquantize)

                Outputsc=torch.zeros(len(Xquantize))
                for i in range(len(Xquantize)):
                    sum=0
                    XBinary=DecimalToBinary(int(Xquantize[i]),XBit)
                    SC=BISC(XBinary)
                    andisSC=0
                    ValW=int(Wquantize[i])
                    while ValW>0:
                        OutUDCounter=UpDownCounter(ValW,SC[(len(SC)-1)-andisSC],1)
                        andisSC+=1
                        ValW=ValW-1#DownCounter
                        sum=sum+OutUDCounter
                    Outputsc[i]=sum*(2**XBit)
                #print("Outputsc",Outputsc)
                OutDequantize=zeropoint_dequantize(Outputsc,zeropoint,scale,summation)
                #print("OutDequantize",OutDequantize)
                output[batch,y] = torch.sum(OutDequantize) + self.biases[y]

        return output

In [124]:
class FCLayer2(nn.Linear):
    def __init__(self, n_inputs, n_outputs):
        super(FCLayer2, self).__init__(n_inputs, n_outputs)
        print("n_inputsfc2=",n_inputs)
        print("n_outputsfc2=",n_outputs)

        self.n_inputs = n_inputs
        self.n_outputs = n_outputs

        # Initialize weights and biases as PyTorch tensors
        self.weights = layer_weightsfc2#nn.Parameter(torch.randn(n_outputs, n_inputs))
        self.biases = layer_biasfc2#nn.Parameter(torch.zeros(n_outputs))

    def forward(self, input_data):
        #////////////////////////////Skippy Multiplicant///////////////////////////////////
        XBit=8

        def UpDownCounter(Enable,x,IncDec):
          IntX=int(x)

          if Enable != 0:
            if IntX == 1:
              if IncDec == 1:
                IntX=1
              else:
                IntX=-1
            else:
              IntX=0
          else:
            IntX=0
          return IntX
        #/////////////////////////////////////////////////////////////////////////////////////////////////////
        def BISC(In):
          LenSc= 2 ** XBit

          sc=np.zeros(LenSc)

          for x in range(XBit):
            i=(2 ** x)-1
            while i<LenSc:
              sc[i] = In[x]
              i+= 2 ** (x+1)

          sc = np.flip(sc)
          return sc

        #/////////////////////////////////////////////////////////////////////////////////////////////////////
        def DecimalToBinary(num,XBit):
          i = 0
          bnum = []
          Binary=''
          while num!=0:
            rem = num%2
            bnum.insert(i, rem)
            i = i+1
            num = int(num/2)
          i = i-1
          while i>=0:
            Binary=Binary+str(bnum[i])
            i = i-1

          while len(Binary)<XBit:
            Binary='0'+Binary

          return Binary

        #///////////////////////////////////////////////////////////////////////////////////////////////////

        output=torch.zeros((input_data.shape[0],self.n_outputs))
        #print("shapeoutputfc1=",output.shape)

        for batch in range(input_data.shape[0]):

            for y in range(self.n_outputs):
                #print("input_data",input_data)
                #print("input_data[batch,:]",input_data[batch,:])
                #print("self.weights",self.weights)
                #print("self.weights[y,:]",self.weights[y,:])

                X=input_data[batch,:]
                W=self.weights[y,:]
                #///////////////////////////Quantize input & Weight//////////////////////////////
                summation= torch.flatten(X + W)
                Xquantize,Wquantize,scale,zeropoint=zeropoint_quantize(X,W)
                #print("Xquantize",Xquantize)
                #print("Wquantize",Wquantize)

                Outputsc=torch.zeros(len(Xquantize))
                for i in range(len(Xquantize)):
                    sum=0
                    XBinary=DecimalToBinary(int(Xquantize[i]),XBit)
                    SC=BISC(XBinary)
                    andisSC=0
                    ValW=int(Wquantize[i])
                    while ValW>0:
                        OutUDCounter=UpDownCounter(ValW,SC[(len(SC)-1)-andisSC],1)
                        andisSC+=1
                        ValW=ValW-1#DownCounter
                        sum=sum+OutUDCounter
                    Outputsc[i]=sum*(2**XBit)
                #print("Outputsc",Outputsc)
                OutDequantize=zeropoint_dequantize(Outputsc,zeropoint,scale,summation)
                #print("OutDequantize",OutDequantize)
                output[batch,y] = torch.sum(OutDequantize) + self.biases[y]

        return output


In [125]:
class MyModel(nn.Module):
    def __init__(self, num_classes):
        super(MyModel, self).__init__()

        self.conv1 =  nn.Conv2d(1, 6, kernel_size=5)
        self.relu1=nn.ReLU()
        self.maxpool1=nn.MaxPool2d(kernel_size = 2, stride = 2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.relu2=nn.ReLU()
        self.maxpool2=nn.MaxPool2d(kernel_size = 2, stride = 2)
        self.conv3 = nn.Conv2d(16, 120, kernel_size=5)

        self.fc1 = FCLayer1(120, 84)
        self.relu4 = nn.ReLU()
        self.fc2 = FCLayer2(84, num_classes)

    def forward(self, x):

        out = self.conv1(x)
        out = self.relu1(out)
        out = self.maxpool1(out)
        out = self.conv2(out)
        out = self.relu2(out)
        out = self.maxpool2(out)
        out = self.conv3(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.relu4(out)
        out = self.fc2(out)

        return out

In [126]:
model1 = MyModel(num_classes)

n_inputsfc1= 120
n_outputsfc1= 84
n_inputsfc2= 84
n_outputsfc2= 10


In [127]:
model1.load_state_dict(torch.load('/content/drive/MyDrive/lenet-weight/weightsLenetPytorch3Conv-2.pt'))

<All keys matched successfully>

In [128]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)

with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images[0:5]
        labels = labels[0:5]
        outputs = model1(images)
        #input("pls enter a code")
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        print("total=",total)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

<ipython-input-115-d432a0e2c37e>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tensor_input = torch.tensor(X)
<ipython-input-115-d432a0e2c37e>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tensor_weight = torch.tensor(W)
<ipython-input-115-d432a0e2c37e>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Mins[0]=torch.tensor(torch.min(tensor_input))
<ipython-input-115-d432a0e2c37e>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True

total= 5
total= 10
total= 15
total= 20
total= 25
total= 30
total= 35
total= 40
total= 45
total= 50
total= 55
total= 60
total= 65
total= 70
total= 75
total= 80
total= 85
total= 90
total= 95
total= 100
total= 105
total= 110
total= 115
total= 120
total= 125
total= 130
total= 135
total= 140
total= 145
total= 150
total= 155
total= 160
total= 165
total= 170
total= 175
total= 180
total= 185
total= 190
total= 195
total= 200
total= 205
total= 210
total= 215
total= 220
total= 225
total= 230
total= 235
total= 240
total= 245
total= 250
total= 255
total= 260
total= 265
total= 270
total= 275
total= 280
total= 285
total= 290
total= 295
total= 300
total= 305
total= 310
total= 315
total= 320
total= 325
total= 330
total= 335
total= 340
total= 345
total= 350
total= 355
total= 360
total= 365
total= 370
total= 375
total= 380
total= 385
total= 390
total= 395
total= 400
total= 405
total= 410
total= 415
total= 420
total= 425
total= 430
total= 435
total= 440
total= 445
total= 450
total= 455
total= 460
total= 4